# Advanced Lane Finding - Project 4 of Udacity's Self-Driving Car Nanodegree

1. Camera Calibration
2. Distortion correction
3. Image inspection
4. Perspective correction

## 1. Camera Calibration

In [ ]:
%run AdvLaneCamera.py
camera = AdvCamera()
camera.chessboard_calibrate_camera()
camera.save_to_pickle()

## 2. Distortion Correction

In [ ]:
%run AdvLaneHelper.py

lane_helper = AdvLaneHelper(camera=camera)

images = glob.glob('camera_cal/calibration*.jpg')

index = 0
col_count = 3
row_count = 7
fig = plt.figure(figsize=(16,32))

# Step through the list and search for chessboard corners
for fname in images:
    img = lane_helper.load_and_undistort(fname)
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.imshow(img)
    index += 1

plt.show()

## 3. Example image inspection

In [ ]:
example_images = lane_helper.get_example_images()

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))

index = 0

example_image = None

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index*2+1)
    plt.imshow(img)
    img = camera.undistort(img)
    plt.imshow(img)
    index += 1

plt.show()

# 4. Perspective correction

In [ ]:
%run AdvLanePerspectiveTransform.py

example_image = lane_helper.load_and_undistort(example_images[6])

perspective_transform = LanePerspectiveTransform(example_image)

# paint trapez into the image
image_copy = np.copy(example_image)

for index in range(4):
    pa = perspective_transform.org_src[index]
    pb = perspective_transform.org_src[(index+1)%4]
    cv2.line(image_copy, pa, pb, (255,0,0), 4)

fig = plt.figure(figsize=(20,20))
plt.imshow(image_copy)

plt.show()

In [ ]:
warped = perspective_transform.transform_perspective_top(example_image)
fig = plt.figure(figsize=(20,20))
plt.imshow(warped)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_video = "project_video.mp4"

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video))

### Video of the project video after distortion and perspective correction

In [ ]:
def process_image(image):
    undistorted = camera.undistort(image)
    warped = perspective_transform.transform_perspective_top(undistorted)
    return warped
    
from_above_video = 'test_videos_output/from_above.mp4'

white_output = from_above_video
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(from_above_video))

## Highlighting lanes

In [ ]:
import ntpath

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = lane_helper.load_and_undistort(fname)
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.title(ntpath.basename(fname))
    warped = perspective_transform.transform_perspective_top(img)
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

In [ ]:
%run AdvLaneThresher.py

thresher = AdvLaneThresher()

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = lane_helper.load_and_undistort(fname)
    img = thresher.create_binary_mask(img)
    warped = perspective_transform.transform_perspective_top(img)
    
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.title(ntpath.basename(fname))
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

# Histograms to highlight likely lane positions

The histograms below show where there are very likely positions for the left and right lane by summing up the positive matches in the binary mask for each column

In [ ]:
col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = lane_helper.load_and_undistort(fname)
    img = thresher.create_binary_mask(img)
    warped = perspective_transform.transform_perspective_top(img)

    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.title(ntpath.basename(fname))
    histogram = np.sum(warped[img.shape[0]//2:,:], axis=0)
    plt.plot(histogram)
    index += 1

plt.show()

# Detection of the lanes

With a sliding window we now try to detect which way each line takes along the top view image.

In case of a single image we assume the left and right lane to start at the histograms peaks.

When processing several images we calculate the likeliness that the new histogram value is valid and otherwise use a more likely starting position from a previous frame.

In [ ]:
%run AdvLaneFinder.py

lane_finder = LaneFinder(camera, perspective_transform, thresher)

for cur_fn in example_images:
    img = cv2.imread(cur_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    lane_finder.clear_history()
    out_img, cam_img, persp = lane_finder.find_lanes_using_window(img)
    
    fig = plt.figure(figsize=(12,8))
    plt.title(cur_fn)
    plt.imshow(out_img)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()    


In [ ]:
%run AdvLaneFinder.py

lane_finder = LaneFinder(camera, perspective_transform, thresher)

for cur_fn in example_images:
    img = cv2.imread(cur_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    lane_finder.clear_history()
    out_img, cam_img, persp = lane_finder.find_lanes_using_window(img)
    
    fig = plt.figure(figsize=(12,8))
    plt.title(cur_fn)
    plt.imshow(cam_img)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()    

# Creating top view videos

Here I verify how the lane detector reacts to a sequence of images and smoothes the variation between single frames

In [ ]:
%run AdvLaneVideoCreator.py

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(find_lanes_raw))

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(find_lanes_raw))

# Visualizing the detected lane lines in the original image

In [ ]:
%run AdvLaneFinder.py

lane_finder = LaneFinder(camera, perspective_transform, thresher)

for cur_fn in example_images:
    img = cv2.imread(cur_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    lane_finder.clear_history()
    out_img, cam_img, persp = lane_finder.find_lanes_using_window(img)
    
    fig = plt.figure(figsize=(12,8))
    plt.title(cur_fn)
    plt.imshow(persp)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()    